In [13]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle
import random
import glob
from sklearn.model_selection import train_test_split
import tensorflow_addons as tfa
from tensorflow.keras import applications
from tensorflow.keras import callbacks
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras import callbacks

DATADIR = "C:/Users/Kajoj/magisterka/dataset"
CATEGORIES = ["benign", "malignant"]
IMG_SIZE = 200
CALLBACK = callbacks.EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=20, restore_best_weights=True)


In [14]:
import pickle

pickle_in = open("X_train_200.pickle", "rb")
X_train = pickle.load(pickle_in)

pickle_in = open("Y_train_200.pickle", "rb")
Y_train = pickle.load(pickle_in)

pickle_in = open("X_test_200.pickle", "rb")
X_test = pickle.load(pickle_in)

pickle_in = open("Y_test_200.pickle", "rb")
Y_test = pickle.load(pickle_in)

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=X_train.shape[1:]),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=None), # No activation on final dense layer
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) 
])


model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tfa.losses.TripletSemiHardLoss())



In [16]:

history = model.fit(
    X_train, Y_train, batch_size=32,
    epochs=50, validation_split = 0.3, callbacks=CALLBACK)

Epoch 1/50
187/187 [==============================] - 127s 676ms/step - loss: 0.8529 - val_loss: 0.8278
Epoch 2/50
187/187 [==============================] - 125s 670ms/step - loss: 0.6799 - val_loss: 0.6740
Epoch 3/50
187/187 [==============================] - 125s 669ms/step - loss: 0.6356 - val_loss: 0.5490
Epoch 4/50
187/187 [==============================] - 127s 679ms/step - loss: 0.5674 - val_loss: 0.5756
Epoch 5/50
187/187 [==============================] - 126s 676ms/step - loss: 0.5243 - val_loss: 0.5459
Epoch 6/50
187/187 [==============================] - 124s 661ms/step - loss: 0.4800 - val_loss: 0.7660
Epoch 7/50
187/187 [==============================] - 124s 662ms/step - loss: 0.4858 - val_loss: 0.6308
Epoch 8/50
187/187 [==============================] - 125s 666ms/step - loss: 0.4554 - val_loss: 0.6710
Epoch 9/50
187/187 [==============================] - 127s 679ms/step - loss: 0.4637 - val_loss: 0.6822
Epoch 10/50
187/187 [==============================] - 124s 666m

In [17]:
train_embs = model.predict(X_train)
test_embs = model.predict(X_test)

115/115 [==============================] - 12s 108ms/step


In [18]:
import io
import tensorflow_datasets as tfds

np.savetxt("vecs.tsv", test_embs, delimiter='\t')

out_m = io.open('meta.tsv', 'w', encoding='utf-8')
[out_m.write(str(x) + "\n") for x in Y_test]
out_m.close()


try:
  from google.colab import files
  files.download('vecs.tsv')
  files.download('meta.tsv')
except:
  pass

In [19]:
knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(train_embs, Y_train)
y_test_pred_knn = knn.predict_proba(test_embs)


In [20]:
predict = []
for pred in y_test_pred_knn:
    if(pred[0]>=pred[1]):
        predict.append(0)
    else:
        predict.append(1)

In [21]:
TP = 0
FP = 0
TN = 0
FN = 0

for i in range (len(predict)):
    if predict[i] == 0 and Y_test[i] == 0:
        TN += 1
    elif predict[i] == 1 and Y_test[i] == 1:
        TP += 1
    elif predict[i] == 0 and Y_test[i] == 1:
        FN += 1
    elif predict[i] == 1 and Y_test[i] == 0:
        FP += 1

In [22]:
czulosc = TP / (TP + FN)
swoistosc = TN / (TN + FP)

print(czulosc)
print(swoistosc)

0.9072052401746725
0.9557618787547788


In [23]:
model.save('simple-model')

INFO:tensorflow:Assets written to: simple-model\assets


INFO:tensorflow:Assets written to: simple-model\assets
